In [2]:
import numpy as np
import pandas as pd
import os, sys

In [3]:
orig_dir = "/Volumes/NexStar3/Preclinical_fMRI_Converted"

def getFiles(path, substring, *args):
    array = []
    for root, dirs, files in os.walk(path):
            for file in files:
                stop = False
                if substring in file:
                    for arg in args: 
                        if arg in file: 
                            stop = True
                    if stop is True: 
                        continue
                    filepath = root + '/' + file
                    array.append(filepath)
                    # print(filepath)
    return array;

In [4]:
os.chdir(orig_dir)

f_artifact = []
t_artifact = []
f_non_artifact = []
t_non_artifact = []
count = 0
for root, dirs, files in os.walk(orig_dir):
    for directory in dirs:
        # make this an option later
        if "Preclin" in directory:
            mriDirs = []
            for d in os.listdir(directory): 
                if "Encoding1_AP_MB4_2_5mmISO_2" in d: 
                    mriDirs.append(d)
            os.chdir(directory)
            for d in mriDirs: 
                # the files that indicate which id's are artifacts
                if "fix_s8.txt" in os.listdir(d):
                    artifact_numbers_file = open(d + "/fix_s8.txt")
                else: 
                    continue
                for d2 in os.listdir(d): 
                    if "melodic_s8" in d2: 
                        melodic_folder = d2
                os.chdir(d)
                # switch into melodic dir 
                line = artifact_numbers_file.readline()
                artifact_numbers_file.close()
                artifact_numbers = line.strip().split(",")
                # every var after "t" is substrings to exclude
                directory = root + '/' + directory + '/' + d + '/' + melodic_folder + '/report'
                f_files = getFiles(directory, "f", "EV", "index", "IC", "._", ".html", "DS", "png", "fix")
                t_files = getFiles(directory, "t", "EV", "index", "IC", "._", "f", ".html", "DS", "png", "fix")
                for file_path in f_files:
                    filename = file_path.split("/")[-1]
                    file_id = filename.split("f")[1]
                    file_id = file_id.split(".")[0]
                    if file_id in artifact_numbers:
                        f_artifact.append(file_path)
                    else:
                        f_non_artifact.append(file_path)
                for file_path in t_files:
                    filename = file_path.split("/")[-1]
                    file_id = filename.split("t")[1]
                    file_id = file_id.split(".")[0]
                    if file_id in artifact_numbers:
                        t_artifact.append(file_path)
                    else:
                        t_non_artifact.append(file_path)
                os.chdir('..')
        os.chdir(root)
        


In [5]:
from sklearn.model_selection import train_test_split
from numpy import median

# load all the text file data
# we're loading both the f and t file data, 
# the first half of the matrix will be the 
# f data and the second half the t data
data = []
labels = []
for f in f_artifact:
    with open(f) as file: 
        temp = []
        for line in file: 
            temp.append(float(line.strip()))
        median = np.median(temp)
        while len(temp) < 282: 
            temp.append(median)
        data.append(temp)
        labels.append(1)

for f in f_non_artifact: 
    with open(f) as file: 
        temp = []
        for line in file: 
            temp.append(float(line.strip()))
        median = np.median(temp)
        while len(temp) < 282: 
            temp.append(median)
        data.append(temp)
        labels.append(0)

    
for t in t_artifact: 
    with open(t) as file: 
        temp = []
        for line in file: 
            temp.append(float(line.strip()))
        median = np.median(temp)
        while len(temp) < 282: 
            temp.append(median)
        data.append(temp)
        labels.append(1)


for t in t_non_artifact: 
    with open(t) as file: 
        temp = []
        for line in file: 
            temp.append(float(line.strip()))
        median = np.median(temp)
        while len(temp) < 282: 
            temp.append(median)
        data.append(temp)
        labels.append(1)

print(len(data))
print(data[0])
print(data[-1])


10100
[1844.005408, 1231.175072, 1973.282303, 1110.607259, 1099.742054, 171.3238988, 46.66925789, 341.365946, 483.0505317, 11.83270465, 163.5344833, 862.3850272, 603.9223705, 121.5948195, 4.323965826, 353.6699343, 107.829319, 13.3688063, 14.43343927, 189.2814329, 80.84162626, 417.3390082, 924.8530283, 163.7093634, 1501.850033, 181.9450836, 1252.364839, 268.891555, 107.7926375, 127.2145208, 576.8446142, 153.9914148, 66.3495204, 73.61947638, 97.29072965, 72.07250148, 680.2686392, 52.82122628, 15.9995106, 58.41033774, 329.0918756, 34.241317, 145.5923751, 24.0497863, 212.0538433, 640.370246, 56.08864774, 1136.531794, 1017.848916, 332.2197187, 333.6734826, 715.7557182, 31.00097021, 39.61136192, 34.42050249, 573.3561317, 327.21444, 367.0643246, 715.7804577, 131.0229739, 7.475960397, 1.491068231, 156.9123192, 207.696072, 440.4249359, 82.68452923, 242.354186, 555.5770656, 219.4476608, 176.84242, 11.3660595, 703.9879937, 971.3364446, 8.642266235, 16.69639337, 520.2381067, 96.64155794, 665.33768

In [6]:
# data = np.array(data)
# labels = np.array(labels)
print(data[-1])
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

[-0.1739767667, 0.4265662671, 0.9657776909, 1.906484651, 1.931189777, 1.222536426, 0.6390196396, 0.1742923816, 0.8880222507, 1.75314631, 2.162263583, 1.755670276, -0.2334944667, -0.9326274534, 0.1703248969, 1.497373871, 1.474298924, 0.7786846725, 0.05626030902, -0.9557243216, -0.7761605339, -0.6844777178, -0.3689201124, 0.4634190906, 1.264733855, 1.520187776, 0.176283833, -0.8226825555, -1.468633168, -0.4443759583, -0.4907779187, 0.5363306399, 1.35675925, 1.631386505, 1.231691324, 0.06131523266, 0.2936769178, 0.02239175976, 1.071402259, 1.005077714, 1.977681594, 1.142900787, 0.4836975059, -1.066952226, -0.8603936035, 0.1977114974, 0.3805449558, 0.7478431218, -0.5759891584, -0.70160386, -1.384522429, -1.426981568, -1.120701106, -1.171278466, -0.9388609913, -1.155894357, -0.531144525, -0.2214875965, 1.131148455, 2.000323348, 1.97481714, 0.3122434415, -0.5411072642, -1.3515589, -0.4627334376, -0.9700546477, -0.142764036, 0.2403345297, 1.497999796, 0.6936494805, 0.9266807102, 0.9490661434,

In [7]:
from sklearn.preprocessing import Imputer

imputer = Imputer(strategy="median")

imputer.fit_transform(X_train)

array([[-2.25440514e-01, -6.46928942e-01,  4.51980571e-01, ...,
         8.13504110e-02,  8.13504110e-02,  8.13504110e-02],
       [ 6.94855883e+02,  2.01088709e+02,  8.73336596e+02, ...,
         2.38298285e+02,  2.38298285e+02,  2.38298285e+02],
       [-2.11897218e+00, -1.86024104e+00, -1.32901315e+00, ...,
         2.02220251e-01,  2.02220251e-01,  2.02220251e-01],
       ...,
       [-4.56762995e-01, -4.17672523e-01, -7.13369562e-01, ...,
        -4.14911475e-02, -4.14911475e-02, -4.14911475e-02],
       [ 1.79324755e+04,  7.12761186e+02,  1.45274816e+03, ...,
         6.05424240e+01,  6.05424240e+01,  6.05424240e+01],
       [-3.04320108e-01, -6.72208795e-01, -5.98390365e-01, ...,
        -3.04435634e-02, -3.04435634e-02, -3.04435634e-02]])

In [16]:
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
# print(X_train.shape)
# print(y_train.shape)

svm_clf = SVC(kernel="poly", degree=3, coef0=1, C=5)
extra_trees_clf = ExtraTreesClassifier(n_estimators=10, random_state=42)
mlp_clf = MLPClassifier(random_state=42)
random_forest_clf = RandomForestClassifier(n_estimators=10, random_state=42)
# random_forest_clf.fit(X_train, y_train)
# y_pred = random_forest_clf.predict(X_test)

estimators = [random_forest_clf, extra_trees_clf, svm_clf, mlp_clf]
for estimator in estimators: 
    print("Training the ", estimator)
    estimator.fit(X_train, y_train)

accuracy_score(y_test, y_pred)


# score = cross_val_score(log_clf, X_train, y_train, cv=5, verbose=3)
# score.mean()

Training the  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)
Training the  ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)
Training the  SVC(C=5, cache_size=200, class_weight=None, coef0=1,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probabilit

0.8400990099009901

In [37]:
from sklearn.model_selection import GridSearchCV

param_distributions = {"n_estimators": [5, 10, 15, 20, 25]}
forest_rnd_cv = GridSearchCV(random_forest_clf, param_distributions, verbose=2, cv=5)
forest_rnd_cv.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] n_estimators=5 ..................................................
[CV] ................................... n_estimators=5, total=   0.4s
[CV] n_estimators=5 ..................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV] ................................... n_estimators=5, total=   0.4s
[CV] n_estimators=5 ..................................................
[CV] ................................... n_estimators=5, total=   0.4s
[CV] n_estimators=5 ..................................................
[CV] ................................... n_estimators=5, total=   0.4s
[CV] n_estimators=5 ..................................................
[CV] ................................... n_estimators=5, total=   0.4s
[CV] n_estimators=10 .................................................
[CV] .................................. n_estimators=10, total=   0.6s
[CV] n_estimators=10 .................................................
[CV] .................................. n_estimators=10, total=   0.6s
[CV] n_estimators=10 .................................................
[CV] .................................. n_estimators=10, total=   0.7s
[CV] n_estimators=10 .................................................
[CV] .

[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   25.5s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [5, 10, 15, 20, 25]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [36]:
param_distributions = {"n_estimators": [5, 10, 15, 20, 25]}
extra_rnd_cv = GridSearchCV(extra_trees_clf, param_distributions, verbose=2, cv=5)
extra_rnd_cv.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] n_estimators=5 ..................................................
[CV] ................................... n_estimators=5, total=   0.2s
[CV] n_estimators=5 ..................................................
[CV] ................................... n_estimators=5, total=   0.2s
[CV] n_estimators=5 ..................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV] ................................... n_estimators=5, total=   0.2s
[CV] n_estimators=5 ..................................................
[CV] ................................... n_estimators=5, total=   0.2s
[CV] n_estimators=5 ..................................................
[CV] ................................... n_estimators=5, total=   0.2s
[CV] n_estimators=10 .................................................
[CV] .................................. n_estimators=10, total=   0.2s
[CV] n_estimators=10 .................................................
[CV] .................................. n_estimators=10, total=   0.2s
[CV] n_estimators=10 .................................................
[CV] .................................. n_estimators=10, total=   0.2s
[CV] n_estimators=10 .................................................
[CV] .................................. n_estimators=10, total=   0.3s
[CV] n_estimators=10 .................................................
[CV] .

[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   10.5s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [5, 10, 15, 20, 25]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [32]:
# param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
# svm_grd_clf = RandomizedSearchCV(svm_clf, param_distributions, verbose=2, cv=5)
# svm_grd_clf.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] C=9.738498583753392, gamma=0.0010637950301769015 ................
[CV] . C=9.738498583753392, gamma=0.0010637950301769015, total=  56.8s
[CV] C=9.738498583753392, gamma=0.0010637950301769015 ................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.0min remaining:    0.0s


[CV] . C=9.738498583753392, gamma=0.0010637950301769015, total= 1.1min
[CV] C=9.738498583753392, gamma=0.0010637950301769015 ................
[CV] . C=9.738498583753392, gamma=0.0010637950301769015, total= 1.0min
[CV] C=9.738498583753392, gamma=0.0010637950301769015 ................
[CV] . C=9.738498583753392, gamma=0.0010637950301769015, total=  49.7s
[CV] C=9.738498583753392, gamma=0.0010637950301769015 ................
[CV] . C=9.738498583753392, gamma=0.0010637950301769015, total=  59.2s
[CV] C=8.430609506703654, gamma=0.0010569331592336275 ................
[CV] . C=8.430609506703654, gamma=0.0010569331592336275, total= 1.2min
[CV] C=8.430609506703654, gamma=0.0010569331592336275 ................
[CV] . C=8.430609506703654, gamma=0.0010569331592336275, total= 1.1min
[CV] C=8.430609506703654, gamma=0.0010569331592336275 ................
[CV] . C=8.430609506703654, gamma=0.0010569331592336275, total= 1.0min
[CV] C=8.430609506703654, gamma=0.0010569331592336275 ................
[CV] .

KeyboardInterrupt: 

In [40]:
forest_rnd_cv.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [43]:
print(forest_rnd_cv.best_score_)
print(extra_rnd_cv.best_score_)

0.851980198019802
0.8527227722772277


In [49]:
print([estimator.score(X_test, y_test) for estimator in estimators])

from sklearn.ensemble import VotingClassifier

named_estimators = [
    ("random_forst_clf", forest_rnd_cv.best_estimator_), 
    ("extra_trees_clf", extra_rnd_cv.best_estimator_), 
#     ("svm_clf", svm_clf), 
#     ("mlp_clf", mlp_clf), 
]

[0.8371287128712871, 0.8341584158415841, 0.8113861386138614, 0.8272277227722772]


In [50]:
voting_clf = VotingClassifier(named_estimators)
voting_clf.fit(X_train, y_train)

voting_clf.score(X_test, y_test)

/Users/mac2researchlab/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.847029702970297

In [22]:
from sklearn.externals import joblib

joblib.dump(voting_clf, "ensemble_clf.pkl")

voting_clf = joblib.load("ensemble_clf.pkl")

In [25]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from scipy.stats import reciprocal, uniform

# param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
param_distributions = {"n_estimators": [5, 10, 15, 20, 25]}
ens_grd_cv = GridSearchCV(voting_clf, param_distributions, verbose=2, cv=5)
ens_grd_cv.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] n_estimators=5 ..................................................


ValueError: Invalid parameter n_estimators for estimator VotingClassifier(estimators=[('random_forst_clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
    ...       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None). Check the list of available parameters with `estimator.get_params().keys()`.

In [11]:
rnd_search_cv.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=15, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [12]:
rnd_search_cv.best_score_

0.85

In [54]:
rnd_search_cv.best_estimator_.fit(X_train, y_train)

y_pred = forest_rnd_cv.best_estimator_.predict(X_test)
accuracy_score(y_test, y_pred)

0.846039603960396

In [55]:
from sklearn.metrics import precision_score, recall_score

print(precision_score(y_pred, y_test))
print(recall_score(y_pred, y_test))

0.7987758224942617
0.9560439560439561


In [58]:
y_pred = extra_rnd_cv.best_estimator_.predict(X_test)

print(precision_score(y_pred, y_test))
print(recall_score(y_pred, y_test))

0.8026013771996939
0.947606142728094


In [61]:
os.chdir("/Users/mac2researchlab/Desktop/brandon_melodic/melodic-classifier")
joblib.dump(extra_rnd_cv.best_estimator_, "extra_grd_cv.pkl")

['extra_grd_cv.pkl']

In [60]:
print(accuracy_score(y_test, y_pred))

0.8435643564356435
